In [1]:
from src.data_preprocessor import DataProcessor
from data_configs.configs import *
from models.decision_tree import DecisionTree, DecisionTreeNode
from models.null_model import NullModelClassification
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
import numpy as np

config = breast_cancer_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)

### Data Load and Preprocessing ###

In [2]:
raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

data_2 = data_1.drop(columns=['Sample code number'])

In [3]:
data_2

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2.0,1,1,1,2
695,2,1,1,1,2,1.0,1,1,1,2
696,5,10,10,3,7,3.0,8,10,2,4
697,4,8,6,4,3,4.0,10,6,1,4


In [4]:
data_train, data_val = cross_validator.random_partition(data_2, random_state=42)

## Decision Tree Performance ##

In [5]:
# Initialize lists to store scores for decision tree, pruned decision tree, and null model
dt_scores = {'zero_one_loss': [], 'f1_score': [], 'precision': [], 'recall': []}
pruned_dt_scores = {'zero_one_loss': [], 'f1_score': [], 'precision': [], 'recall': []}
null_model_scores = {'zero_one_loss': [], 'f1_score': [], 'precision': [], 'recall': []}

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=True)):
    train_data = train_set.drop(columns=config['target_column'])
    train_target = train_set[config['target_column']]
    test_features = test_set.drop(columns=config['target_column'])
    test_true_vals = test_set[config['target_column']]

    # Decision Tree Model for classification
    decision_tree = DecisionTree(config, data_2)
    decision_tree.root = decision_tree.build_classification_tree(train_data, train_target)
    predictions = decision_tree.predict(test_features)

    # Calculate and store decision tree scores
    scores = Evaluation.calculate_classification_scores(test_true_vals, predictions)
    for key in dt_scores:
        dt_scores[key].append(scores[key])

    # Pruning the Decision Tree
    decision_tree.prune(decision_tree.root, data_val)  # Ensure data_val is correctly defined as your validation set
    pruned_predictions = decision_tree.predict(test_features)

    # Calculate and store pruned decision tree scores
    pruned_scores = Evaluation.calculate_classification_scores(test_true_vals, pruned_predictions)
    for key in pruned_dt_scores:
        pruned_dt_scores[key].append(pruned_scores[key])

    # Null Model for classification
    null_model = NullModelClassification(config=config)
    null_model_prediction = null_model.naive_classifier(test_set)

    # Calculate and store null model scores
    null_scores = Evaluation.calculate_classification_scores(test_true_vals, null_model_prediction)
    for key in null_model_scores:
        null_model_scores[key].append(null_scores[key])

# Calculate average scores for each model
average_dt_scores = {metric: np.mean(values) for metric, values in dt_scores.items()}
average_pruned_dt_scores = {metric: np.mean(values) for metric, values in pruned_dt_scores.items()}
average_null_model_scores = {metric: np.mean(values) for metric, values in null_model_scores.items()}

# Print average scores
print("Average Decision Tree Scores:")
for metric, avg_score in average_dt_scores.items():
    print(f"{metric}: {avg_score}")

print("\nAverage Pruned Decision Tree Scores:")
for metric, avg_score in average_pruned_dt_scores.items():
    print(f"{metric}: {avg_score}")

print("\nAverage Null Model Scores:")
for metric, avg_score in average_null_model_scores.items():
    print(f"{metric}: {avg_score}")


Average Decision Tree Scores:
zero_one_loss: 0.06689964157706094
f1_score: 0.932940773451184
precision: 0.9342020854276927
recall: 0.9331003584229391

Average Pruned Decision Tree Scores:
zero_one_loss: 0.055444188428059385
f1_score: 0.9447897406666181
precision: 0.9455921357853677
recall: 0.9445558115719408

Average Null Model Scores:
zero_one_loss: 0.35062724014336916
f1_score: 0.5113278649043025
precision: 0.421685374674015
recall: 0.6493727598566308
